In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('../scrapedData/xendan-5.csv')

In [ ]:
df.head()

In [ ]:
df['Article'][0].replace('\n','').replace('\u200c',' ').replace('خه ندان-',' ')

In [1]:
import concurrent.futures
from tqdm.notebook import tqdm
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
data_list = []
partial_save_amount = 5

def scrape_xendan(lastId):
    try:
        url = f"https://www.xendan.org/detailnews/{lastId}"
        proxied_fetch = f'https://spontaneous-faloodeh-9e9321.netlify.app/?destination={url}'
        html = urlopen(proxied_fetch)
        bs = BeautifulSoup(html, 'html.parser')    
        header_elem = bs.find('div', {'class': "detail-top"})
        article_elem = bs.find('div', {'class': "detail-big-text-p"})
        img_elem = bs.find('div', {'class': "detail-big-img"})
        if header_elem and article_elem and header_elem.find('h1'):
            header = header_elem.find('h1').text.strip()
            article = article_elem.text.strip()
            image = img_elem.img['src'] if img_elem else ""
            data_list.append({'Header': header, 'Article': article, 'Link': url, 'Image': image})
            #print(f"Scraped: {url}")
            if len(data_list) % 1000 == 0:
                print(f"saved at: {len(data_list)}")
                dff=pd.DataFrame(data_list)
                dff.to_csv(f'../scrapedData/xendan-{len(data_list)}.csv')
                time.sleep(20)
        else:
            print(f"Skipping: {url}")
    except Exception as e:
        print(f'Error at {lastId}: {e}')
def main():
    max_threads = 10  # Adjust the number of threads as needed
    last_id = 154955  # Set the last ID to start scraping
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = []
        for i in tqdm(range(last_id, -1, -1)):
            futures.append(executor.submit(scrape_xendan, i))
        for future in concurrent.futures.as_completed(futures):
            pass  # Optionally process results here

if __name__ == "__main__":
    main()


  0%|          | 0/154956 [00:00<?, ?it/s]

165552